This module provides set of classes for working with WildApricot public API v2.
Public API documentation can be found here: http://help.wildapricot.com/display/DOC/API+Version+2
Example:
    api = WaApi.WaApiClient()
    api.authenticate_with_contact_credentials("admin@youraccount.com", "your_password")
    accounts = api.execute_request("/v2/accounts")
    for account in accounts:
        print(account.PrimaryDomainName)
       

In [4]:
import datetime
import urllib.request
import urllib.response
import urllib.error
import urllib.parse
import json
import base64


class WaApiClient(object):
    """Wild apricot API client."""
    auth_endpoint = "https://oauth.wildapricot.org/auth/token"
    api_endpoint = "https://api.wildapricot.org"
    _token = None
    client_id = None
    client_secret = None

    def __init__(self, client_id, client_secret):
        self.client_id = client_id
        self.client_secret = client_secret

    def authenticate_with_apikey(self, api_key, scope=None):
        """perform authentication by api key and store result for execute_request method
        api_key -- secret api key from account settings
        scope -- optional scope of authentication request. If None full list of API scopes will be used.
        """
        scope = "auto" if scope is None else scope
        data = {
            "grant_type": "client_credentials",
            "scope": scope
        }
        encoded_data = urllib.parse.urlencode(data).encode()
        request = urllib.request.Request(self.auth_endpoint, encoded_data, method="POST")
        request.add_header("ContentType", "application/x-www-form-urlencoded")
        request.add_header("Authorization", 'Basic ' + base64.standard_b64encode(('APIKEY:' + api_key).encode()).decode())
        response = urllib.request.urlopen(request)
        self._token = WaApiClient._parse_response(response)
        self._token.retrieved_at = datetime.datetime.now()

    def authenticate_with_contact_credentials(self, username, password, scope=None):
        """perform authentication by contact credentials and store result for execute_request method
        username -- typically a contact email
        password -- contact password
        scope -- optional scope of authentication request. If None full list of API scopes will be used.
        """
        scope = "auto" if scope is None else scope
        data = {
            "grant_type": "password",
            "username": username,
            "password": password,
            "scope": scope
        }
        encoded_data = urllib.parse.urlencode(data).encode()
        request = urllib.request.Request(self.auth_endpoint, encoded_data, method="POST")
        request.add_header("ContentType", "application/x-www-form-urlencoded")
        auth_header = base64.standard_b64encode((self.client_id + ':' + self.client_secret).encode()).decode()
        request.add_header("Authorization", 'Basic ' + auth_header)
        response = urllib.request.urlopen(request)
        self._token = WaApiClient._parse_response(response)
        self._token.retrieved_at = datetime.datetime.now()

    def execute_request(self, api_url, api_request_object=None, method=None):
        """
        perform api request and return result as an instance of ApiObject or list of ApiObjects
        api_url -- absolute or relative api resource url
        api_request_object -- any json serializable object to send to API
        method -- HTTP method of api request. Default: GET if api_request_object is None else POST
        """
        if self._token is None:
            raise ApiException("Access token is not abtained. "
                               "Call authenticate_with_apikey or authenticate_with_contact_credentials first.")

        if not api_url.startswith("http"):
            api_url = self.api_endpoint + api_url

        if method is None:
            if api_request_object is None:
                method = "GET"
            else:
                method = "POST"

        request = urllib.request.Request(api_url, method=method)
        if api_request_object is not None:
            request.data = json.dumps(api_request_object, cls=_ApiObjectEncoder).encode()

        request.add_header("Content-Type", "application/json")
        request.add_header("Accept", "application/json")
        request.add_header("Authorization", "Bearer " + self._get_access_token())
        print(f'Sending: {request.data}')

        try:
            response = urllib.request.urlopen(request)
            print(f'Received: {response.read().decode()}')
            return WaApiClient._parse_response(response)
        except urllib.error.HTTPError as httpErr:
            if httpErr.code == 400:
                raise ApiException(httpErr.read())
            else:
                raise

    def _get_access_token(self):
        expires_at = self._token.retrieved_at + datetime.timedelta(seconds=self._token.expires_in - 100)
        now = datetime.datetime.now()
        if datetime.datetime.now() > expires_at:
            self._refresh_auth_token()
        return self._token.access_token

    def _refresh_auth_token(self):
        data = {
            "grant_type": "refresh_token",
            "refresh_token": self._token.refresh_token
        }
        encoded_data = urllib.parse.urlencode(data).encode()
        request = urllib.request.Request(self.auth_endpoint, encoded_data, method="POST")
        request.add_header("ContentType", "application/x-www-form-urlencoded")
        auth_header = base64.standard_b64encode((self.client_id + ':' + self.client_secret).encode()).decode()
        request.add_header("Authorization", 'Basic ' + auth_header)
        response = urllib.request.urlopen(request)
        self._token = WaApiClient._parse_response(response)
        self._token.retrieved_at = datetime.datetime.now()

    @staticmethod
    def _parse_response(http_response):
        decoded = json.loads(http_response.read().decode())
        if isinstance(decoded, list):
            result = []
            for item in decoded:
                result.append(ApiObject(item))
            return result
        elif isinstance(decoded, dict):
            return ApiObject(decoded)
        else:
            return None


class ApiException(Exception):
    def __init__(self, value):
        self.value = value

    def __str__(self):
        return repr(self.value)


class ApiObject(object):
    """Represent any api call input or output object"""

    def __init__(self, state):
        self.__dict__ = state
        for key, value in vars(self).items():
            if isinstance(value, dict):
                self.__dict__[key] = ApiObject(value)
            elif isinstance(value, list):
                new_list = []
                for list_item in value:
                    if isinstance(list_item, dict):
                        new_list.append(ApiObject(list_item))
                    else:
                        new_list.append(list_item)
                self.__dict__[key] = new_list

    def __str__(self):
        return json.dumps(self.__dict__)

    def __repr__(self):
        return json.dumps(self.__dict__)


class _ApiObjectEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, ApiObject):
            return obj.__dict__
        # Let the base class default method raise the TypeError
        return json.JSONEncoder.default(self, obj)

In [5]:
wa_api = WaApiClient("FunRunner", "dhp41we7yh5xkp2frgaxrq4c900ueh")

In [6]:
wa_api.authenticate_with_apikey("lh2j12qchscwrpdnpr06p28oruc9w7")

In [7]:
accounts = wa_api.execute_request("/v2/accounts")

Sending: None
Received: [{"Id":30507,"Url":"https://api.wildapricot.org/v2/Accounts/30507","PrimaryDomainName":"Aarc.wildapricot.org","Resources":[{"Name":"Features","Url":"https://api.wildapricot.org/v2/accounts/30507/Features/","AllowedOperations":["GET"]},{"Name":"Contacts","Url":"https://api.wildapricot.org/v2/accounts/30507/Contacts/","AllowedOperations":["GET","POST","PUT","DELETE"]},{"Name":"Membership levels","Url":"https://api.wildapricot.org/v2/accounts/30507/MembershipLevels/","AllowedOperations":["GET"]},{"Name":"Contact fields","Url":"https://api.wildapricot.org/v2/accounts/30507/ContactFields/","AllowedOperations":["GET","POST","PUT","DELETE"]},{"Name":"Member groups","Url":"https://api.wildapricot.org/v2/accounts/30507/MemberGroups/","AllowedOperations":["GET"]},{"Name":"Saved searches","Url":"https://api.wildapricot.org/v2/accounts/30507/SavedSearches/","AllowedOperations":["GET"]},{"Name":"Bundles","Url":"https://api.wildapricot.org/v2/accounts/30507/Bundles/","Descrip

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [8]:
type(accounts)

NameError: name 'accounts' is not defined

In [ ]:
type(accounts[0])

In [9]:
wa_api.client_id

'FunRunner'

In [10]:
type(wa_api._token)

__main__.ApiObject

In [11]:
wa_api.response

AttributeError: 'WaApiClient' object has no attribute 'response'

In [14]:
import pickle
contacts = pickle.load(open('wa_response.p', 'rb'))

In [15]:
type(contacts)


list

In [17]:
len(contacts)

219

In [18]:
type(contacts[0])

chalicelib.wa_api.ApiObject

In [19]:
response = contacts

In [20]:
type(response[0])

chalicelib.wa_api.ApiObject

In [21]:
contacts = [contact.__dict__ for contact in response]

In [22]:
type(contacts[0])

dict

In [23]:
len(contacts)

219

In [24]:
for contact in contacts[0:10]:
    print(f'Is {contact["FirstName"]} a member? {"Yes" if contact["Status"] == "Active" else "No"}' )

Is Chuks a member? Yes
Is Hugh a member? Yes
Is Vincent a member? Yes
Is Jim a member? Yes
Is Bonnie a member? Yes
Is Richard a member? Yes
Is Paul a member? Yes
Is Bob a member? Yes
Is Lisa a member? Yes
Is Janice a member? Yes


In [26]:
contacts[0]["Status"] = "Inactive"
for contact in contacts[0:10]:
    print(f'Is {contact["FirstName"]} a member? {"Yes" if contact["Status"] == "Active" else "No"}' )
contacts[0]["Status"] = "Active"

Is Chuks a member? No
Is Hugh a member? Yes
Is Vincent a member? Yes
Is Jim a member? Yes
Is Bonnie a member? Yes
Is Richard a member? Yes
Is Paul a member? Yes
Is Bob a member? Yes
Is Lisa a member? Yes
Is Janice a member? Yes


In [30]:
import pickle
api_response = pickle.load(open('wa_all_contacts.p', 'rb'))
all_contacts = [contact.__dict__ for contact in api_response]

In [44]:
len(all_contacts)

1298

In [33]:
type(all_contacts)

list

In [36]:
type(all_contacts[0])

dict

In [41]:
print(all_contacts[0].keys())

dict_keys(['FirstName', 'LastName', 'Email', 'DisplayName', 'Organization', 'ProfileLastUpdated', 'MembershipLevel', 'MembershipEnabled', 'Status', 'FieldValues', 'Id', 'Url', 'IsAccountAdministrator', 'TermsOfUseAccepted'])


In [45]:
for contact in all_contacts[0:10]:
    print(contact['Status'])

Lapsed
Active


KeyError: 'Status'

In [46]:
members = [contact for contact in all_contacts if "Status" in contact and contact["Status"] == "Active"]
len(members)

267